# 동화 텍스트 생성 AI

## 검증 -> 통합 프롬프트 재구성 -> LLM 생성 로직 구현

1단계: 라이브러리 설치 및 설정

In [42]:
# 1. 라이브러리 설치
!pip install openai -q

import os
from openai import OpenAI
import json
from getpass import getpass

# 2. API Key 입력 (실행 시 입력창에 sk-로 시작하는 키를 넣고 엔터를 치세요)
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key를 입력하세요: ")

client = OpenAI()

print("✅ 설정이 완료되었습니다.")

OpenAI API Key를 입력하세요: ··········
✅ 설정이 완료되었습니다.


2단계: 동화 생성 엔진 (아키텍처 반영 로직)

In [43]:
import json

def generate_fairy_tale(user_input):
    print(f"\n🚀 [프로세스 시작] 사용자 입력: \"{user_input}\"")

    # ------------------------------------------------------------------
    # Step 1: 입력값 검증 및 분석 (Safety & Requirement Check)
    # ------------------------------------------------------------------
    print("1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...")

    check_prompt = f"""
    당신은 동화 서비스의 '안전 관리자'입니다.
    사용자의 입력이 동화를 만들기에 적절한지 판단하세요.
    폭력적, 선정적, 혐오 표현이 있거나 동화와 전혀 관련 없는 요청이라면 'FALSE'를,
    적절하다면 'TRUE'를 출력하세요.
    설명 없이 오직 TRUE 또는 FALSE만 대답하세요.

    사용자 입력: {user_input}
    """

    check_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a safety moderator."},
                  {"role": "user", "content": check_prompt}],
        temperature=0.0
    )

    is_safe = check_response.choices[0].message.content.strip()

    if "FALSE" in is_safe:
        return "⚠️ [차단됨] 부적절하거나 동화 생성과 관련 없는 요청입니다. 다시 입력해주세요."

    print("   -> ✅ 검증 통과.")


    # ------------------------------------------------------------------
    # Step 2: 통합 프롬프트 재구성 (Prompt Re-engineering)
    # 사용자의 짧은 입력을 상세한 '동화 기획서'로 변환합니다.
    # ------------------------------------------------------------------
    print("2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...")

    architect_prompt = f"""
    당신은 '한국형 동화 기획자'입니다.
    사용자 입력을 분석하여 시대 배경을 정하고, 그 시대에 맞지 않는 설정은 **반드시** 해당 시대에 어울리는 개념으로 바꿔야 합니다.

    [필수 수행 절차]
    1. **시대 결정(Era)**: [단군시대, 조선시대, 근대, 현대] 중 택 1
       - 입력에 시대 언급이 없으면, 내용에 가장 잘 어울리는 시대를 고르되 기본은 '현대'입니다.

    2. **시대별 개념 치환 (Context Mapping)**:
       **만약 '현대(Modern)'가 선택되었다면:**
       - 왕, 왕비 -> '시장님', '교장 선생님', '회장님', '부모님'
       - 공주, 왕자 -> '인기 아이돌', '부잣집 친구', '반장', '전학생'
       - 무도회 -> '학예회', '학교 축제', '생일 파티', '장기자랑'
       - 왕국, 성 -> '학교', '우리 동네', '아파트', '놀이공원'
       - 마법 -> '과학 기술', '신기한 도구', '스마트폰', '꿈속 상상' (혹은 현대 판타지로 처리)

       **만약 '조선시대'가 선택되었다면:**
       - 파티 -> '잔치', '연회'
       - 빵/케이크 -> '떡', '다과'
       - 드레스 -> '비단 한복', '당의'

    [사용자 입력]
    {user_input}

    [출력 포맷 - JSON 형식만 출력할 것]
    {{
      "title": "동화 제목",
      "era": "선택한 시대 (단군시대/조선시대/근대/현대)",
      "characters": "주인공과 등장인물 상세 설정 (이름도 한국식으로: 돌쇠, 점순이, 호야 등)",
      "setting": "배경 설정 (구체적인 한국 지명 포함)",
      "korean_items": ["이야기에 사용할 한국적 소재1", "소재2", "소재3"],
      "tone": "분위기",
      "moral": "교훈",
      "plot_outline": "줄거리 요약"
    }}
    """

    plan_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a creative story architect. Output JSON only."},
                  {"role": "user", "content": architect_prompt}],
        response_format={ "type": "json_object" },
        temperature=0.7
    )

    story_plan = json.loads(plan_response.choices[0].message.content)

    # ------------------------------------------------------------------
    # [중간 로직] 시대별 오프닝 문구 매핑 (Python Logic)
    # ------------------------------------------------------------------
    era_map = {
        "단군시대": "단군 할아버지가 터를 잡으실 적에,",
        "조선시대": "호랑이가 곰방대를 피울 적에,",
        "근대": "전차 소리가 땡땡 울리던 시절,",
        "현대": "백마를 타고 오는 초인이 왔을 적에,"
    }
    # 기획된 era에 맞는 오프닝을 가져오고, 없으면 기본값(조선시대) 사용
    selected_era = story_plan.get("era", "조선시대")
    opening_sentence = era_map.get(selected_era, "호랑이가 곰방대를 피울 적에,")

    print("\n   📄 [생성된 동화 설계도]")
    print(f"   - 제목: {story_plan['title']}")
    print(f"   - 시대: {selected_era} -> 첫 문장: \"{opening_sentence}\"")
    print(f"   - 한국적 소재: {story_plan.get('korean_items')}")


    # ------------------------------------------------------------------
    # Step 3: 동화 텍스트 생성 (LLM Generation)
    # ------------------------------------------------------------------
    print("\n3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...")

    writer_prompt = f"""
    당신은 한국의 정서를 담아 따뜻하고 교훈적인 이야기를 만드는 베스트셀러 동화 작가입니다.
    아래의 [동화 설계도]를 바탕으로 아이들이 읽기 좋은 동화를 작성해 주세요.

    [작성 가이드]
    1. **첫 문장 필수**: 이야기는 반드시 다음 문장으로 시작해야 합니다.
       -> "{opening_sentence}"
    2. **한국적 요소 활용**: 기획된 한국적 소재({story_plan.get('korean_items')})를 이야기에 자연스럽게 녹여내세요.
       (예: 파티 대신 잔치, 빵 대신 떡, 왕자님 대신 도령님 사용)
    3. 문체: '~했어요', '~였답니다'와 같은 부드러운 구어체(경어)를 사용하세요.
    4. 구성: 도입, 전개, 위기, 절정, 결말의 구조를 갖추세요.
    5. 분량: 읽어주었을 때 3분~5분 정도의 길이 (약 1500자 내외).
    6. 어휘: '파티' 대신 '잔치/축제', '드레스' 대신 '한복/예쁜 옷' 등 한국적 어휘 사용.
    7. **설정 준수**:
       - 시대가 '{story_plan['era']}'입니다.
       - 절대로 이 시대에 맞지 않는 단어(예: 현대에 '공주님', 조선시대에 '스마트폰')를 쓰지 마세요.


    [동화 설계도]
    {json.dumps(story_plan, ensure_ascii=False)}
    """

    story_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a warm and creative fairy tale writer."},
                  {"role": "user", "content": writer_prompt}],
        temperature=0.8
    )

    final_story = story_response.choices[0].message.content

    return final_story, story_plan

# 함수 테스트
print("✅ 동화 생성 엔진 준비 완료 (한국적 요소 및 시대별 도입부 적용)")

✅ 동화 생성 엔진 준비 완료 (한국적 요소 및 시대별 도입부 적용)


## 생성 → 후처리 검증(유해성/저작권/품질) → (미통과 시) 피드백 루프 → 최종 포맷팅 구현

3단계: 후처리 검증 및 재생성

In [44]:
import json

# ==============================================================================
# Step 4: 후처리 검증 (Editor AI)
# ==============================================================================
import json
from openai import OpenAI

# client = OpenAI(...) # 클라이언트 초기화 부분은 기존 코드 유지

def verify_story_quality(story_text, story_plan):
    print("\n🧐 [검증 단계] '수석 편집장 AI'가 루브릭에 맞춰 원고를 정밀 심사 중입니다...")

    # 사용자가 입력한 교훈 (없을 경우 대비)
    target_moral = story_plan.get('moral', '교훈 없음')

    # 상세 루브릭이 포함된 시스템 프롬프트
    verification_prompt = f"""
    당신은 5~8세 아동용 동화 서비스 'Once Upon a Time'의 [수석 품질 검수자]입니다.
    작가(AI)가 작성한 동화 원고를 아래 [검증 루브릭]에 따라 엄격하게 1~5점 척도로 채점하고, 최종 판정을 내리세요.

    [검증 루브릭 (Scoring Rubric)]


    1. 유해성 (Safety) - Critical
    - 기준: 아동에게 정서적 충격, 공포, 혐오감을 주는가?
    - 5점: 부정적 요소 전무, 심리적 안정감을 주는 따뜻한 내용.
    - 4점: 갈등이 있으나 권선징악으로 건전하게 해소됨.
    - 3점: 약간의 거친 표현(예: "멍청이", 가벼운 다툼)이 있으나 맥락상 허용 가능.
    - 2점: [Fail] 따라 할 수 있는 나쁜 행동, 구체적인 공포 묘사 포함.
    - 1점: [Fail] 잔혹성, 선정성, 혐오 표현, 트라우마 유발.

    2. 저작권 (Copyright) - Critical
    - 기준: 기존 유명 IP(디즈니, 해리포터 등)의 고유명사나 설정을 도용했는가?
    - 5점: 고유명사, 플롯, 캐릭터가 완전히 오리지널함.
    - 4점: 전형적 클리셰(왕자, 공주)는 있으나 특정 작품이 연상되지 않음.
    - 3점: 구조가 유명 동화와 흡사하지만 고유명사는 다름.
    - 2점: [Fail] 특정 IP(예: 엘사, 호그와트, 피카츄)를 연상시키는 명칭이나 설정 등장.
    - 1점: [Fail] 기존 작품의 이름, 대사, 스토리를 그대로 베낌.

    3.  품질: 교훈 적합성 (Alignment)
    - 기준: 사용자(부모)가 입력한 [요청 교훈]이 이야기에 잘 녹아있는가?
    - 5점: 교훈이 결말과 완벽히 연결되며 아이가 스스로 깨달을 수 있음.
    - 4점: 교훈이 잘 드러나나, 다소 직접적인 설명 방식임.
    - 3점: 교훈이 포함되어 있으나 이야기 흐름과 약간 겉돔.
    - 2점: [Warning] 교훈이 억지스럽거나 미미함.
    - 1점: [Warning] 요청된 교훈과 무관한 내용.

    4.  품질: 정서 및 표현 (Aesthetics)
    - 기준: 한국적 정서(존댓말)와 아동 눈높이 표현(의성어/의태어)이 적절한가?
    - 5점: 부드러운 '해요체', 풍부한 의성어/의태어, 따뜻한 어조.
    - 4점: 문장이 매끄럽고 이해하기 쉬운 단어 사용.
    - 3점: 문법은 맞으나 딱딱하거나 교과서적인 문체.
    - 2점: [Warning] 번역투 문장, 어려운 한자어 사용.
    - 1점: [Warning] 비문이 많거나 문맥이 끊김.

    [원고 내용]
    {story_text}

    [출력 포맷 - 반드시 JSON 형식만 출력]
    {{
        "scores": {{
            "safety": Integer (1-5),
            "copyright": Integer (1-5),
            "alignment": Integer (1-5),
            "aesthetics": Integer (1-5)
        }},
        "reasons": {{
            "safety": "점수 부여 사유 (한 문장)",
            "copyright": "점수 부여 사유",
            "alignment": "점수 부여 사유",
            "aesthetics": "점수 부여 사유"
        }},
        "final_decision": "PASS" 또는 "FAIL",
        "rewrite_suggestion": "FAIL인 경우, 작가에게 전달할 구체적인 수정 지시사항 (PASS면 null)"
    }}
    ** 주의: 유해성이나 저작권 점수가 3점 미만이면 무조건 'final_decision'은 "FAIL"입니다. **
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a strict editor evaluator. Output JSON only."},
                {"role": "user", "content": verification_prompt}
            ],
            response_format={ "type": "json_object" },
            temperature=0.0 # 평가의 일관성을 위해 0으로 설정
        )

        result = json.loads(response.choices[0].message.content)

        # 로그 출력 (디버깅용)
        print(f"   -> 판정 결과: {result['final_decision']}")
        print(f"   -> 점수: Safety({result['scores']['safety']}), Copyright({result['scores']['copyright']}), Quality({result['scores']['alignment']}/{result['scores']['aesthetics']})")

        return result

    except Exception as e:
        print(f"검증 중 오류 발생: {e}")
        # 오류 발생 시 안전을 위해 FAIL 처리하거나 예외 처리
        return {"final_decision": "FAIL", "rewrite_suggestion": "시스템 오류로 검증 실패"}

# ==============================================================================
# Step 5: 피드백 반영 재작성 (Refinement)
# ==============================================================================
def refine_story(previous_story, feedback, story_plan):
    print(f"\n✍️ [수정 단계] 작가 AI가 피드백을 반영하여 원고를 다시 쓰고 있습니다...")

    refine_prompt = f"""
    당신은 동화 작가입니다. 초안이 편집 심사에서 반려되었습니다.
    아래 피드백을 반영하여 동화를 다시 작성해주세요.

    [기획 정보]
    제목: {story_plan['title']}
    교훈: {story_plan['moral']}

    [편집장 피드백 (수정 요청)]
    {feedback}

    [기존 원고]
    {previous_story}

    위 내용을 바탕으로 피드백을 해결한 완성된 동화를 다시 써주세요.
    (문체와 분량은 기존 가이드를 따르세요.)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a professional fairy tale writer."},
                  {"role": "user", "content": refine_prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content

# ==============================================================================
# Step 6: 최종 포맷팅 (Formatter)
# ==============================================================================
def format_final_output(valid_story, story_plan):
    # DB 스키마에 맞춘 최종 JSON 구조 생성
    final_data = {
        "title": story_plan['title'],
        "content": valid_story,
        "summary": story_plan['plot_outline'],
        "category": "창작동화",
        "moral": story_plan['moral'],
        "characters": story_plan['characters'],
        "tags": [story_plan['tone'], "교훈", "어린이"],
        "verification_status": "Verified"
    }
    return final_data

4단계: 메인 실행기-전체 프로세스 통합 및 출력 제어

In [45]:
import json

# ==============================================================================
# [Helper] 점수표 출력 함수 (시각적 효과)
# ==============================================================================
def print_score_card(attempt, result):
    scores = result.get('scores', {})
    reasons = result.get('reasons', {})
    decision = result.get('final_decision', 'UNKNOWN')

    print(f"\n📊 [검증 리포트 - 시도 {attempt}회차]")
    print("-" * 50)
    print(f"1. 유해성 (Safety)    : {'★' * scores.get('safety', 0)} ({scores.get('safety', 0)}/5)")
    print(f"2. 저작권 (Copyright) : {'★' * scores.get('copyright', 0)} ({scores.get('copyright', 0)}/5)")
    print(f"3. 교훈성 (Alignment) : {'★' * scores.get('alignment', 0)} ({scores.get('alignment', 0)}/5)")
    print(f"4. 심미성 (Aesthetics): {'★' * scores.get('aesthetics', 0)} ({scores.get('aesthetics', 0)}/5)")
    print("-" * 50)
    print(f"🧐 종합 판정: [{'✅ PASS' if decision == 'PASS' else '❌ FAIL'}]")

    if decision == 'FAIL':
        print(f"⚠️ 사유: {reasons.get('safety') or reasons.get('copyright') or '기준 미달'}")
    print("=" * 60)


# ==============================================================================
# [메인 실행기] 전체 프로세스 통합
# ==============================================================================
def run_process_with_display(user_input):
    max_retries = 3
    attempt = 0
    current_story = ""
    story_plan = {}
    feedback = None
    is_finalized = False

    print(f"\n🚀 [System] 동화 생성 프로세스를 시작합니다: 주제 '{user_input}'")

    # -------------------------------------------------------
    # 생성 및 검증 반복 루프 (Retry Loop)
    # -------------------------------------------------------
    while attempt < max_retries:
        attempt += 1

        # 1. 스토리 생성 (초안 vs 재작성안)
        if attempt == 1:
            print(f"\n📝 [Step 1] 초안 작성 중... (Attempt {attempt})")
            current_story, story_plan = generate_fairy_tale(user_input)

            print("\n" + "┌" + "─"*58 + "┐")
            print("│  📜 [초안 (Draft Version)]                           │")
            print("└" + "─"*58 + "┘")
            print(current_story)
            print("-" * 60)

        else:
            print(f"\n🔄 [Step 1-Retry] 피드백 반영하여 재작성 중... (Attempt {attempt})")
            print(f"   (반영할 피드백: {feedback})")
            current_story, story_plan = generate_fairy_tale(user_input, feedback=feedback)

            print("\n" + "┌" + "─"*58 + "┐")
            print(f"│  📜 [수정안 {attempt} (Revised Draft)]                       │")
            print("└" + "─"*58 + "┘")
            print(current_story)
            print("-" * 60)

        # 2. 검증 수행 (Step 2)
        print("\n🔍 [Step 2] 품질 검증 수행 중...")
        verification_result = verify_story_quality(current_story, story_plan)

        # 3. 점수표 출력 (요청하신 부분)
        print_score_card(attempt, verification_result)

        # 4. 결과에 따른 분기 처리
        decision = verification_result.get('final_decision', 'FAIL')

        if decision == "PASS":
            # (옵션) 문체 스타일 교정
            scores = verification_result.get('scores', {})
            if scores.get('aesthetics', 0) < 4: # 4점 미만이면 스타일 다듬기
                print("\n✨ [Step 3] 내용은 통과했으나, 문체를 더 부드럽게 다듬습니다 (Polishing)...")
                current_story = refine_style(current_story)
            else:
                print("\n✅ [Step 3] 수정 없이 최종안으로 확정합니다.")

            is_finalized = True
            break

        else:
            # FAIL 처리
            feedback = verification_result.get('rewrite_suggestion', '내용 수정 필요')
            if attempt < max_retries:
                print(f"🚫 [System] 기준 미달로 다시 작성합니다.")
            else:
                print("❌ [System] 수정 한도를 초과했습니다.")

    # -------------------------------------------------------
    # 최종 결과 출력 (Final Output)
    # -------------------------------------------------------
    if is_finalized:
        print("\n" + "="*60)
        print("🎉 [Final Result] 최종 완성된 동화")
        print("="*60)
        print(current_story)
        print("="*60)

        # JSON 변환
        final_json = format_final_output(current_story, story_plan)
        return final_json
    else:
        print("\n⚠️ 최종 생성 실패.")
        return None



실행 테스트

In [46]:
# 테스트 실행
if __name__ == "__main__":
    user_topic = input("동화 주제를 입력하세요: ")
    if user_topic:
        run_process_with_display(user_topic)

동화 주제를 입력하세요: 조선시대 공주님이 성밖을 몰래 나오는 이야기

🚀 [System] 동화 생성 프로세스를 시작합니다: 주제 '조선시대 공주님이 성밖을 몰래 나오는 이야기'

📝 [Step 1] 초안 작성 중... (Attempt 1)

🚀 [프로세스 시작] 사용자 입력: "조선시대 공주님이 성밖을 몰래 나오는 이야기"
1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...
   -> ✅ 검증 통과.
2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...

   📄 [생성된 동화 설계도]
   - 제목: 조선의 비밀 모험
   - 시대: 조선시대 -> 첫 문장: "호랑이가 곰방대를 피울 적에,"
   - 한국적 소재: ['비단 한복', '시장 거리의 한과', '효종갱']

3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...

┌──────────────────────────────────────────────────────────┐
│  📜 [초안 (Draft Version)]                           │
└──────────────────────────────────────────────────────────┘
"호랑이가 곰방대를 피울 적에," 옛날 옛적 조선시대 한양에 효림이라는 공주가 살고 있었어요. 효림은 성 안에서의 생활이 답답했어요. 매일 같은 나날이 반복되고, 궁궐 밖 세상은 너무나 멀게만 느껴졌지요. 그러던 어느 날, 효림은 평소 아끼던 비단 한복을 입고 조심스레 성곽을 빠져나왔답니다.

한양의 성곽 밖은 효림에게 새로운 세상이었어요. 시장 거리는 사람들로 북적였고, 각양각색의 한과들이 진열된 가게에서 달콤한 냄새가 났어요. 효림은 그 냄새를 따라가다 시장의 아들 민수를 만나게 되었어요. 민수는 밝고 명랑한 성격으로, 시장 거리의 모든 가게와 사람들에 대해 아는 것이 많았답니다.

"안녕하세요, 저는 민수예요. 공주님은 여기 처음 오셨나요?" 민수가 물었어요. 효